In [9]:
import pandas as pd

from mongo_wrapper.mongo_wrapper import MongoWrapper

import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
MONGO_IP = os.getenv("MONGO_IP")
MONGO_PORT = os.getenv("MONGO_PORT")
MONGO_DB = "news_outlets"
MONGO_USERNAME = os.getenv("MONGO_USERNAME")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")

In [4]:
mongo_client = MongoWrapper(
    db=MONGO_DB,
    user=MONGO_USERNAME,
    password=MONGO_PASSWORD,
    ip=MONGO_IP,
    port=MONGO_PORT
)

2025-05-12 10:50:01,451 WARNING Logger Mongo was configured with True console stream
2025-05-12 10:50:01,888 INFO Connected to news_outlets database on 206.81.16.39


In [6]:
mediacloud_desempleo_collection = mongo_client.get_collection_entries(collection="mediacloud_desempleo")

In [11]:
data = pd.DataFrame(mediacloud_desempleo_collection)

In [12]:
data.head()

,_id,language,media_name,media_url,publish_date,title,url,content
0,3aac93101bf98af51d6dd813d4f3c2d2f3ff4df0762d99...,es,siempre.mx,siempre.mx,2022-01-21,Un año de presidente,http://www.siempre.mx/2022/01/un-ano-de-presid...,A un año de ejercer la presidencia de los Esta...
1,7f91a8bae3b694f0723fded92cf368f99e0ddce8adb8ad...,es,elsoldechilpancingo.mx,elsoldechilpancingo.mx,2022-01-25,"“La calor” y “una mala alimentación”, causas d...",https://www.elsoldechilpancingo.mx/2022/01/25/...,- La alcaldesa de Acapulco consideró que esos ...
2,22a79fdb52079465989eb8cfec75329c266b279a45d085...,es,diarioeyipantla.com,diarioeyipantla.com,2022-01-25,Diario Eyipantla Milenio,https://www.diarioeyipantla.com/descarrilada-4t/,Lo único que el gobierno de Veracruz ha ofreci...
3,a99e04f9cc07bf8693c2a42c6cfc7596e2e00282b6446a...,es,elheraldodechihuahua.com.mx,elheraldodechihuahua.com.mx,2022-01-25,Cita Comisión de Trabajo y Previsión Social a ...,https://www.elheraldodechihuahua.com.mx/local/...,Cita Comisión de Trabajo y Previsión Social a ...
4,92107c41b45288fbd7ddcb77cdfc00cd0dc10dca0d63e5...,es,noreste.net,noreste.net,2022-01-25,Carlos Jesús R. – Del Río Virgen podría salir ...,https://noreste.net/carlos-jesus-r-del-rio-vir...,NaN


In [13]:
data["content"].iloc[0]

'A un año de ejercer la presidencia de los Estados Unidos, Joe Biden, no ha logrado superar la profunda polarización social que priva en un país que vive una compleja situación económica que, desde hace 40 no se registraba, con una inflación del 7 por ciento y, un desempleo superior a los 11 millones. Por si esto no fuera suficiente, una pandemia que no ha dado tregua, con más de 68 millones de contagios y cerca de 859 mil fallecimientos.\nSin duda que Biden nunca llegó a imaginar que la aprobación a su trabajo fuera tan baja, sólo un 40 por ciento, lo reconocen.\nEn el balance a un año el presidente, Joe Biden, dijo que su cargo ha tenido “desafíos” pero también “enormes progresos” y, aseguró que no previó una obstrucción republicana tan fuerte contra su gobierno, reconoció también los pasos en falso dados ante una pandemia que está en su apogeo, pero elogió los avances en materia económica…”ha sido un año de desafíos, pero también ha sido un año de enormes progresos”, así es como res